In [2]:
import os
# import time
#
# import matplotlib.pyplot as plt
# import numpy as np
# import torch
# from torch import nn
from torch.utils.data import DataLoader, Subset
# from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
# import torchvision.utils as tvutils
import mmcv
import decord

# import cv2
# import PIL
# import matplotlib.pyplot as plt

from my_utils.vis import *
# from my_utils import models
# from my_utils.utils import check_accuracy
from my_utils.data.dataset import raw_png_processor, raw_video_processor, MyDataset, speed_checker
from my_utils.data.loader import load_frames, load_avi, load_mat
# from my_utils.data.preprocess import sub_mean, reduce

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)  # set default size of plots

# life save magic code
%load_ext autoreload
%autoreload 2

In [3]:
print(f"torch version: {torch.__version__}")
use_cuda = torch.cuda.is_available()
if use_cuda:
    GPU_nums = torch.cuda.device_count()
    GPU = torch.cuda.get_device_properties(0)
    print(f"There are {GPU_nums} GPUs in total.\nThe first GPU is: {GPU}")
    if '3060' in GPU.name:
        print(f"CUDA version: {torch.cuda_version}")
    else:
        print(f"CUDA version: {torch.version.cuda}")
device = torch.device(f"cuda:0" if use_cuda else "cpu")
print(f"Using {device} now!")

torch version: 1.12.0+cu113
There are 1 GPUs in total.
The first GPU is: _CudaDeviceProperties(name='NVIDIA A100-SXM4-80GB', major=8, minor=0, total_memory=81251MB, multi_processor_count=108)
CUDA version: 11.3
Using cuda:0 now!


In [6]:
dp = raw_png_processor(dataset_root='/mnt/lustre/wangyihao/nlos_raw_pngs', mode='train')

In [ ]:
dp.check_data(resume=False)

In [5]:
dp.build_mat_dataset(modal='video', noise_factor=0, resize=(128, 128))

100%|██████████| 1600/1600 [00:23<00:00, 67.89it/s]


In [11]:
dp.fix_video_dataset()

100%|██████████| 1600/1600 [10:33<00:00,  2.53it/s]


In [ ]:
dp.frames2video()

In [8]:
dp.build_dataset_list(data_type='avi')

100%|██████████| 1600/1600 [02:58<00:00,  8.98it/s]


In [9]:
sc = speed_checker()

In [13]:
sc.check(method='avi', times=100)

100%|██████████| 100/100 [00:33<00:00,  2.99it/s]

0.3350698184967041 sec are used to load a video in average.


In [81]:
frame_list = sorted([f for f in os.listdir('/mnt/lustre/wangyihao/nlos_raw_pngs/train/Lying down_p2_r0_white_tiling_07-15-03-16/') if f.endswith('.png')])
start_idx = int(frame_list[0][:-4])

In [82]:
start_idx

7

In [13]:
# decord.bridge.set_bridge('torch')
I420_avi_tensor = load_avi('/mnt/lustre/wangyihao/nlos_raw_pngs/train/Lying down_p2_r0_white_tiling_07-15-03-16/video_I420_N0.avi',
                      output_size=(128, 128), sub_mean=True)

torch.float32


In [79]:
XVID_avi_tensor = load_avi('/mnt/lustre/wangyihao/nlos_raw_pngs/train/Lying down_p2_r0_white_tiling_07-15-03-16/XVID_video_128_N0.avi',
                      output_size=(128, 128))

In [66]:
png_tensor = load_frames('/mnt/lustre/wangyihao/nlos_raw_pngs/train/Lying down_p2_r0_white_tiling_07-15-03-16/',
                         output_size=(128, 128))

In [78]:
I420_avi_tensor

tensor([[[[73, 74, 75,  ..., 37, 37, 35],
          [74, 70, 75,  ..., 37, 37, 38],
          [75, 74, 74,  ..., 37, 37, 35],
          ...,
          [61, 63, 63,  ..., 38, 37, 34],
          [65, 61, 60,  ..., 35, 35, 37],
          [66, 61, 62,  ..., 35, 38, 37]],

         [[73, 74, 75,  ..., 37, 37, 35],
          [74, 70, 75,  ..., 37, 37, 38],
          [75, 74, 74,  ..., 37, 37, 35],
          ...,
          [62, 63, 65,  ..., 38, 37, 34],
          [66, 61, 60,  ..., 35, 35, 37],
          [66, 61, 61,  ..., 35, 38, 37]],

         [[72, 74, 76,  ..., 37, 37, 35],
          [73, 70, 75,  ..., 37, 37, 38],
          [75, 74, 74,  ..., 37, 37, 37],
          ...,
          [61, 63, 63,  ..., 38, 37, 34],
          [65, 61, 60,  ..., 35, 35, 37],
          [66, 61, 62,  ..., 35, 38, 37]],

         ...,

         [[72, 74, 76,  ..., 37, 37, 35],
          [74, 72, 76,  ..., 38, 38, 38],
          [75, 74, 74,  ..., 37, 37, 37],
          ...,
          [62, 65, 65,  ..., 38, 37, 

In [80]:
XVID_avi_tensor

tensor([[[[72, 73, 75,  ..., 37, 37, 37],
          [74, 72, 76,  ..., 37, 37, 37],
          [75, 74, 74,  ..., 37, 37, 37],
          ...,
          [61, 63, 65,  ..., 38, 37, 35],
          [63, 61, 60,  ..., 35, 37, 38],
          [63, 61, 61,  ..., 35, 38, 38]],

         [[72, 73, 75,  ..., 37, 37, 37],
          [74, 72, 76,  ..., 37, 37, 37],
          [75, 74, 74,  ..., 37, 37, 37],
          ...,
          [61, 63, 65,  ..., 38, 37, 35],
          [63, 61, 60,  ..., 35, 37, 38],
          [63, 61, 61,  ..., 35, 38, 38]],

         [[72, 73, 75,  ..., 37, 37, 37],
          [74, 72, 76,  ..., 37, 37, 37],
          [75, 74, 74,  ..., 37, 37, 37],
          ...,
          [61, 63, 65,  ..., 38, 37, 35],
          [63, 61, 60,  ..., 35, 37, 38],
          [63, 61, 61,  ..., 35, 38, 38]],

         ...,

         [[72, 73, 75,  ..., 37, 37, 37],
          [74, 72, 76,  ..., 37, 37, 37],
          [75, 74, 74,  ..., 37, 37, 37],
          ...,
          [63, 65, 65,  ..., 38, 37, 

In [69]:
png_tensor.permute(3, 0, 1, 2)

tensor([[[[74., 77., 78.,  ..., 39., 39., 38.],
          [77., 70., 80.,  ..., 39., 39., 41.],
          [78., 77., 76.,  ..., 40., 40., 38.],
          ...,
          [64., 68., 68.,  ..., 40., 40., 37.],
          [69., 62., 61.,  ..., 38., 38., 40.],
          [70., 64., 65.,  ..., 38., 41., 38.]],

         [[74., 77., 78.,  ..., 39., 39., 38.],
          [78., 70., 79.,  ..., 39., 39., 41.],
          [78., 77., 76.,  ..., 40., 40., 38.],
          ...,
          [64., 68., 68.,  ..., 40., 40., 37.],
          [69., 62., 61.,  ..., 38., 39., 40.],
          [70., 64., 64.,  ..., 38., 41., 38.]],

         [[74., 77., 78.,  ..., 39., 39., 38.],
          [77., 71., 80.,  ..., 39., 39., 41.],
          [78., 77., 76.,  ..., 40., 40., 38.],
          ...,
          [63., 68., 68.,  ..., 40., 40., 37.],
          [69., 62., 61.,  ..., 38., 38., 40.],
          [69., 64., 65.,  ..., 38., 41., 38.]],

         ...,

         [[72., 77., 79.,  ..., 40., 39., 38.],
          [78., 71., 8